In [ ]:
import pandas as pd
import numpy as np

In [ ]:
!pip install --upgrade setuptools

Requirement already up-to-date: setuptools in c:\dev\anaconda3\lib\site-packages (56.2.0)


In [ ]:
conda install -c conda-forge/label/cf202003 scikit-surprise

Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [ ]:
from surprise import Dataset, Reader, KNNWithMeans
from surprise import SVD, accuracy
from surprise.model_selection import GridSearchCV, train_test_split

In [ ]:
df = pd.read_csv('C:/Users/STU/Desktop/appliances_no_null_values.csv')
df.head()

,rating,reviewer_id,product_id,unix_review_time,category,description,also_buy,brand,main_cat,price,review_text,rating_class,time
0,5,A2OXDRWBASV91Y,B00004SQHD,1366416000,"['Appliances', 'Parts & Accessories', 'Range P...",['Coleman Cable 09045 5-Foot Range Cord Made o...,[],Coleman Cable,Tools & Home Improvement,$1.41,complete package I like the fact that the wire...,good,2013-04-20
1,5,A2KG6AWJSWILPR,B00004SQHD,1363392000,"['Appliances', 'Parts & Accessories', 'Range P...",['Coleman Cable 09045 5-Foot Range Cord Made o...,[],Coleman Cable,Tools & Home Improvement,$1.41,Perfect Fit Needed another couple of feet with...,good,2013-03-16
2,5,A2CBE6VYOARZN4,B00004SQHD,1487030400,"['Appliances', 'Parts & Accessories', 'Range P...",['Coleman Cable 09045 5-Foot Range Cord Made o...,[],Coleman Cable,Tools & Home Improvement,$1.41,Everything works Good quality at a reasonable ...,good,2017-02-14
3,5,AVKOTZD5ZIOX5,B00004SQHD,1482278400,"['Appliances', 'Parts & Accessories', 'Range P...",['Coleman Cable 09045 5-Foot Range Cord Made o...,[],Coleman Cable,Tools & Home Improvement,$1.41,50 amp cable Planning on hooking this to my RV,good,2016-12-21
4,5,A33DUVUIC7G553,B00004SQHD,1460678400,"['Appliances', 'Parts & Accessories', 'Range P...",['Coleman Cable 09045 5-Foot Range Cord Made o...,[],Coleman Cable,Tools & Home Improvement,$1.41,"Good range cord, standard nema 14-50p fits wit...",good,2016-04-15


In [ ]:
df[df['product_id'].isnull()]

,rating,reviewer_id,product_id,unix_review_time,category,description,also_buy,brand,main_cat,price,review_text,rating_class,time


In [ ]:
df = df[['reviewer_id', 'product_id', 'rating']]

In [ ]:
df.head()

,reviewer_id,product_id,rating
0,A2OXDRWBASV91Y,B00004SQHD,5
1,A2KG6AWJSWILPR,B00004SQHD,5
2,A2CBE6VYOARZN4,B00004SQHD,5
3,AVKOTZD5ZIOX5,B00004SQHD,5
4,A33DUVUIC7G553,B00004SQHD,5


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 446474 entries, 0 to 446473
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   reviewer_id  446474 non-null  object
 1   product_id   446474 non-null  object
 2   rating       446474 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 10.2+ MB


In [ ]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['reviewer_id', 'product_id', 'rating']], reader)

In [ ]:
from surprise import SVD, SVDpp, SlopeOne, NMF, NormalPredictor, KNNBasic, KNNBaseline, KNNWithMeans, KNNWithZScore, BaselineOnly, CoClustering
from surprise.model_selection import cross_validate

In [ ]:
# SVDpp 최적 파라미터 찾기
param_grid = {
    "n_epochs": [5, 10, 20, 50],
    "lr_all": [0.001, 0.002, 0.005, 0.01],
    "reg_all": [0.2, 0.4, 0.6]
}
# GridSearchCV
gs = GridSearchCV(SVDpp, param_grid, measures=['rmse', 'mae'], cv=3)

In [ ]:
# gs 피팅
gs.fit(data)
print(gs.best_score["rmse"]), print(gs.best_params["rmse"])
print(gs.best_score["mae"]), print(gs.best_params["mae"])
# rmse:1.1648616211300336
# {'n_epochs': 50, 'lr_all': 0.005, 'reg_all': 0.4}
# mse: 0.825855904743514
# {'n_epochs': 50, 'lr_all': 0.01, 'reg_all': 0.2}

1.1648616211300336
{'n_epochs': 50, 'lr_all': 0.005, 'reg_all': 0.4}
0.825855904743514
{'n_epochs': 50, 'lr_all': 0.01, 'reg_all': 0.2}


(None, None)

In [ ]:
trainset, testset = train_test_split(data, test_size=0.1)

# SVD 알고리즘 피팅
algo_svdpp = SVDpp(n_epochs=50, lr_all=0.005, reg_all=0.4)
algo_svdpp.fit(trainset)

In [ ]:
predictions = algo_svdpp.test(testset)
accuracy.rmse(predictions)
# RMSE: 1.1429

RMSE: 1.1429


1.1428749832426746

In [ ]:
# 가상의 id
uid = 'A18C6CLQ8LQKEF'
iid = 'B000WLEL84'
algo_svdpp.predict(uid, iid)

Prediction(uid='A18C6CLQ8LQKEF', iid='B000WLEL84', r_ui=None, est=4.376150373544768, details={'was_impossible': False})

In [ ]:
# UID를 인풋으로 받고 상품을 추천하는 방법은?
# 실험대상: 'A2OXDRWBASV91Y' (실제 reviewer_id 중 아무거나 뽑음)
product_id = df['product_id'].unique()

In [ ]:
uid = 'A2OXDRWBASV91Y'
pred_list = []
top_items = 20
for iid in product_id:
    pred = algo_svdpp.predict(uid, iid)
    pred_list.append((pred[1], pred[3]))
def takeSecond(elem):
    return elem[1]
pred_list.sort(reverse=True, key=takeSecond)
reclist = pred_list[:top_items]
for i in range(top_items):
    print(reclist[i][0])

B017AFO242
B00F4KOVYW
B00GTRBD1S
B00G1968H8
B01BTY5AR8
B00DZUAC8M
B00M04EKXM
B01ARKTLSE
B00DZUA7U0
B0193DQ0T4
B00M0ZDV2W
B00AF7V7HI
B00438PNRK
B015GAWJ84
B00LQDAYT2
B00AF7UWAQ
B01G50YTGE
B00K94PQR4
B0050O204A
B00ECV51DY


In [ ]:
# 기존의 SVD 버전도 시도(파라미터 예전거 그대로)
# SVD 알고리즘 피팅
algo_svd = SVD(n_epochs=50, lr_all=0.01, reg_all=0.2)
algo_svd.fit(trainset)

In [ ]:
predictions = algo_svd.test(testset)
accuracy.rmse(predictions)
# RMSE: 1.1370

RMSE: 1.1370


1.1369677851859699

In [ ]:
# 가상의 id
uid = 'A18C6CLQ8LQKEF'
iid = 'B000WLEL84'
algo_svd.predict(uid, iid)

Prediction(uid='A18C6CLQ8LQKEF', iid='B000WLEL84', r_ui=None, est=4.376150373544768, details={'was_impossible': False})

In [ ]:
# UID를 인풋으로 받고 상품을 추천하는 방법은?
# 실험대상: 'A2KG6AWJSWILPR' (실제 reviewer_id 중 아무거나 뽑음)
product_id = df['product_id'].unique()

In [ ]:
uid = 'A2KG6AWJSWILPR'
pred_list = []
top_items = 20
for iid in product_id:
    pred = algo_svd.predict(uid, iid)
    pred_list.append((pred[1], pred[3]))
def takeSecond(elem):
    return elem[1]
pred_list.sort(reverse=True, key=takeSecond)
reclist = pred_list[:top_items]
for i in range(top_items):
    print(reclist[i][0])

B001DPRHC0
B002YTOCP4
B0036IGT6M
B004H3XVXE
B0050O204A
B005B9CFFS
B00AF7V7HI
B00AF7UUA8
B00AF7RI6M
B00AFSMJ6A
B00AH5ZMVK
B00AO62FYO
B00AOG6HK2
B00AZMGPRA
B00D2J72PM
B00DM8JR2Y
B00DM8KIH2
B00DM8MBVS
B00DZFD10I
B00EBIM09G
